In [1]:
%run "data_generation.ipynb"

生成全尺度数据：

In [3]:
# 生成 60 x 60 的方形结构，力和位移边界条件随机，共100个
ds_full = generate_dataset(60,100)

1 structure generated
2 structure generated
3 structure generated
4 structure generated
5 structure generated
6 structure generated
7 structure generated
8 structure generated
9 structure generated
10 structure generated
11 structure generated
12 structure generated
13 structure generated
14 structure generated
15 structure generated
16 structure generated
17 structure generated
18 structure generated
19 structure generated
20 structure generated
21 structure generated
22 structure generated
23 structure generated
24 structure generated
25 structure generated
26 structure generated
27 structure generated
28 structure generated
29 structure generated
30 structure generated
31 structure generated
32 structure generated
33 structure generated
34 structure generated
35 structure generated
36 structure generated
37 structure generated
38 structure generated
39 structure generated
40 structure generated
41 structure generated
42 structure generated
43 structure generated
44 structure generat

生成缩尺数据并将其分块：

In [3]:
BLOCK_SIZE = 2 # 分块大小（n x n）
ds_red = generate_reduced_dataset(ds_full.dens_full.values, ds_full.bc_cond.values, ds_full.f_cond.values, factor=BLOCK_SIZE)
ds_block = generate_block_dataset(ds_full, ds_red, block_size=BLOCK_SIZE)

保存至Data文件夹下：

In [ ]:
dir = "Data"
ds_full.to_netcdf(dir+"/full.nc")
ds_block.to_netcdf(dir+"/block.nc")